<a href="https://colab.research.google.com/github/aksamiftikhar/aksamiftikhar/blob/main/CIFAR_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow model for CIFAR data classification

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers


print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.8.0



## Load a dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the sample data from integers to floating-point numbers:

In [2]:
cifar = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar.load_data()
print(x_train.dtype, type(x_train), x_train.shape, y_train.shape, x_train.max())

x_train, x_test = x_train / 255.0, x_test / 255.0
print(x_train.dtype, type(x_train), x_train.shape, y_train.shape, x_train.max())


170508288/170498071 [==============================] - 2s 0us/step
uint8 <class 'numpy.ndarray'> (50000, 32, 32, 3) (50000, 1) 255
float64 <class 'numpy.ndarray'> (50000, 32, 32, 3) (50000, 1) 1.0


## Build a custom machine learning model

Build a `tf.keras.Sequential` model by stacking layers.

In [14]:
# model = tf.keras.models.Sequential([
#   tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
#   tf.keras.layers.Dense(256, activation='relu'),
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dropout(0.3),
#   tf.keras.layers.Dense(64, activation='relu'),
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Dense(10)
# ])
model = tf.keras.Sequential()
model.add(layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='glorot_uniform'))
# model.add(layers.MaxPool2D())
# model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='glorot_uniform'))
model.add(layers.AveragePooling2D())
model.add(layers.Dropout(0.3))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='glorot_uniform'))
model.add(layers.AveragePooling2D())
# model.add(layers.Dropout(0.2))
model.add(layers.BatchNormalization())

# model.add(layers.Flatten(input_shape=(32,32,3)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(64))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10))


### Before Training:

Test the raw output of designed CNN model. Almost all classes have equal probability for all the 10 classes, i.e. 0.1. 

In [15]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.00510827, -0.0927939 ,  0.11784013, -0.11720359, -0.0007416 ,
        -0.00041782,  0.07692049, -0.03688416,  0.05850714, -0.07126338]],
      dtype=float32)

Define a loss function for training using `losses.SparseCategoricalCrossentropy`, which takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [16]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [17]:
loss_fn(y_train[:1], predictions).numpy()

2.2221608

Before starting the training, configure and compile the model using Keras `Model.compile`. S

In [18]:
model.compile(optimizer=tf.keras.optimizers.SGD(0.01, momentum=0.9), 
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [19]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1563/1563 [==============================] - 24s 14ms/step - loss: 1.6483 - accuracy: 0.4122
Epoch 2/10
1563/1563 [==============================] - 23s 14ms/step - loss: 1.2965 - accuracy: 0.5402
Epoch 3/10
1563/1563 [==============================] - 23s 14ms/step - loss: 1.1018 - accuracy: 0.6160
Epoch 4/10
1563/1563 [==============================] - 22s 14ms/step - loss: 0.9790 - accuracy: 0.6607
Epoch 5/10
1563/1563 [==============================] - 23s 14ms/step - loss: 0.8708 - accuracy: 0.7010
Epoch 6/10
1563/1563 [==============================] - 23s 14ms/step - loss: 0.7954 - accuracy: 0.7288
Epoch 7/10
1563/1563 [==============================] - 23s 14ms/step - loss: 0.7372 - accuracy: 0.7453
Epoch 8/10
1563/1563 [==============================] - 22s 14ms/step - loss: 0.6962 - accuracy: 0.7614
Epoch 9/10
1563/1563 [==============================] - 22s 14ms/step - loss: 0.6445 - accuracy: 0.7788
Epoch 10/10
1563/1563 [==============================] - 22s 14m

Model evaluation using the test set.

In [20]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 2s - loss: 0.8173 - accuracy: 0.7447 - 2s/epoch - 8ms/step


[0.8172644972801208, 0.744700014591217]

The classifier gives ~75% test accuracy. The training epochs can be increased to further improve the accuracy.